<a href="https://colab.research.google.com/github/Luca4Spreafico/CHALLENGE-2---Ibuprofen/blob/main/Luca_Ibuprofen2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🌐 **Kaggle setup**

In [ ]:
working_dir = "/kaggle/working"
input_dir = "/kaggle/input"
dataset_dir = "/kaggle/input"

## 🌐 **Google Drive setup**

In [1]:
# ___________________________________________________________________________________ collegamento colab
from google.colab import drive

drive.mount("/gdrive")
working_dir = "/gdrive/My Drive/B University/Artificial Networks/an2dl2526c2"
input_dir = working_dir

%cd $working_dir

print(f"Working at folder: {working_dir}")
print(f"Input at folder: {input_dir}")

Mounted at /gdrive
/gdrive/My Drive/B University/Artificial Networks/an2dl2526c2
Working at folder: /gdrive/My Drive/B University/Artificial Networks/an2dl2526c2
Input at folder: /gdrive/My Drive/B University/Artificial Networks/an2dl2526c2


# Set up

In [4]:
# ___________________________________________________________________________________ Libraries

# Set seed for reproducibility
SEED = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(SEED)
random.seed(SEED)

# Import PyTorch
import torch
torch.manual_seed(SEED)
from torch import nn
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader

# Configurazione di TensorBoard e directory
logs_dir = "tensorboard"
!pkill -f tensorboard
%load_ext tensorboard
!mkdir -p models

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

# Import other libraries
import copy
import shutil
from itertools import product
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.gridspec as gridspec

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

PyTorch version: 2.9.0+cu126
Device: cuda


Data Loading

In [7]:
# _________________________________________  DATA LOADING

dataset_dir = input_dir   # adjust to your real folder

train_data_dir = os.path.join(dataset_dir, "train_data")
train_labels_path = os.path.join(dataset_dir, "train_labels.csv")


# ------------------ Check directories (FAIL-FAST) ------------------
print("Validating dataset structure...")

# Check training directory - CRITICAL
if not os.path.exists(train_data_dir):
    raise FileNotFoundError(f"CRITICAL ERROR: Training data folder not found at: {train_data_dir}")
print(f"✓ Training folder found: {train_data_dir}")

# ------------------ Check labels file (FAIL-FAST) ------------------
if not os.path.exists(train_labels_path):
    raise FileNotFoundError(f"CRITICAL ERROR: train_labels.csv not found at: {train_labels_path}")
print(f"✓ Labels file found: {train_labels_path}")

# ------------------ Load labels (FAIL-FAST) ------------------
try:
    df_labels = pd.read_csv(train_labels_path)
    print(f"✓ Loaded {len(df_labels)} training labels.")
except Exception as e:
    raise RuntimeError(f"CRITICAL ERROR: Failed to load labels file: {e}")

# Validate that labels DataFrame is not empty
if len(df_labels) == 0:
    raise ValueError("CRITICAL ERROR: Labels file is empty!")

# Validate that 'label' column exists
if 'label' not in df_labels.columns:
    raise ValueError(f"CRITICAL ERROR: 'label' column not found in CSV. Available columns: {df_labels.columns.tolist()}")

# ------------------ Load training images & masks ------------------
print("\nLoading training images and masks…")

file_list = sorted(os.listdir(train_data_dir))
N = len(df_labels)  # should be 1412

# FAIL-FAST: Check expected file count
if len(file_list) != 2 * N:
    raise ValueError(f"CRITICAL ERROR: Expected {2*N} files, found {len(file_list)}. Dataset is incomplete!")

# First half = images
image_files = file_list[:N]

# Second half = masks
mask_files = file_list[N:N*2]

# Initialize lists to store loaded data as numpy arrays
train_images = []      # List of numpy arrays (variable dimensions)
train_masks = []       # List of numpy arrays (variable dimensions)
train_targets = []     # List of labels
train_dimensions = []  # List to track (height, width) of each sample

# Track validation issues
failed_samples = []
dimension_mismatches = []

# Define label encoding
label_mapping = {
      'Luminal A': 0,
      'Luminal B': 1,
      'HER2(+)': 2,
      'Triple negative': 3
  }

# Iterate through each row in the labels DataFrame
for idx, row in df_labels.iterrows():
    filename_img  = image_files[idx]
    filename_mask = mask_files[idx]
    label         = row["label"]

    img_path  = os.path.join(train_data_dir, filename_img)
    mask_path = os.path.join(train_data_dir, filename_mask)

    try:
        # Open image and mask
        img  = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # Convert to numpy arrays immediately for memory efficiency
        img_array = np.array(img)    # Shape: (H, W, 3)
        mask_array = np.array(mask)  # Shape: (H, W)

        # VALIDATION: Check that image and mask have matching dimensions
        if img_array.shape[:2] != mask_array.shape:
            dimension_mismatches.append({
                'index': idx,
                'filename_img': filename_img,
                'filename_mask': filename_mask,
                'img_shape': img_array.shape,
                'mask_shape': mask_array.shape
            })
            print(f"WARNING: Dimension mismatch at index {idx}: image {img_array.shape} vs mask {mask_array.shape}")
            continue  # Skip this sample

        # Encode the label
        encoded_label = label_mapping[label]

        # Store numpy arrays instead of PIL images
        train_images.append(img_array)
        train_masks.append(mask_array)
        train_targets.append(encoded_label)  # Store encoded label instead
        train_dimensions.append(img_array.shape[:2])


    except Exception as e:
        failed_samples.append({'index': idx, 'error': str(e)})
        print(f"ERROR loading sample index {idx}: {e}")

print(f"\n✓ Successfully loaded {len(train_images)} training samples.")




# ------------------ VALIDATION REPORT ------------------
print("\n" + "="*60)
print("VALIDATION REPORT")
print("="*60)

# Report failed samples
if failed_samples:
    print(f"\n⚠ WARNING: {len(failed_samples)} samples failed to load:")
    for fail in failed_samples[:5]:  # Show first 5
        print(f"  - Index {fail['index']}: {fail['error']}")
    if len(failed_samples) > 5:
        print(f"  ... and {len(failed_samples) - 5} more")

# Report dimension mismatches
if dimension_mismatches:
    print(f"\n⚠ WARNING: {len(dimension_mismatches)} samples had dimension mismatches:")
    for mismatch in dimension_mismatches[:5]:  # Show first 5
        print(f"  - Index {mismatch['index']}: img {mismatch['img_shape']} vs mask {mismatch['mask_shape']}")
    if len(dimension_mismatches) > 5:
        print(f"  ... and {len(dimension_mismatches) - 5} more")

# Check if too many samples failed
total_failed = len(failed_samples) + len(dimension_mismatches)
if total_failed > N * 0.05:  # More than 5% failed
    raise RuntimeError(f"CRITICAL ERROR: Too many samples failed ({total_failed}/{N}). Dataset quality is too low!")

# Validate label distribution
print(f"\nLabel distribution:")
unique_labels, counts = np.unique(train_targets, return_counts=True)
for label, count in zip(unique_labels, counts):
    print(f"  Label {label}: {count} samples ({100*count/len(train_targets):.1f}%)")

# Display dimension statistics
print(f"\nImage dimension statistics:")
heights = [dim[0] for dim in train_dimensions]
widths = [dim[1] for dim in train_dimensions]
print(f"  Height range: {min(heights)} - {max(heights)} pixels (mean: {np.mean(heights):.1f})")
print(f"  Width range: {min(widths)} - {max(widths)} pixels (mean: {np.mean(widths):.1f})")

# Count unique dimensions
unique_dims = set(train_dimensions)
print(f"  Number of unique dimension combinations: {len(unique_dims)}")
if len(unique_dims) <= 5:
    print(f"  Unique dimensions found:")
    for dim in sorted(unique_dims):
        count = train_dimensions.count(dim)
        print(f"    - {dim}: {count} samples")

print("\n" + "="*60)
print("✓ Training data validation complete!")
print("="*60)




Validating dataset structure...
✓ Training folder found: /gdrive/My Drive/B University/Artificial Networks/an2dl2526c2/train_data
✓ Labels file found: /gdrive/My Drive/B University/Artificial Networks/an2dl2526c2/train_labels.csv
✓ Loaded 1412 training labels.

Loading training images and masks…

✓ Successfully loaded 1412 training samples.

VALIDATION REPORT

Label distribution:
  Label 0: 414 samples (29.3%)
  Label 1: 445 samples (31.5%)
  Label 2: 397 samples (28.1%)
  Label 3: 156 samples (11.0%)

Image dimension statistics:
  Height range: 1024 - 4233 pixels (mean: 1341.4)
  Width range: 1024 - 3970 pixels (mean: 1252.3)
  Number of unique dimension combinations: 639

✓ Training data validation complete!


Key Improvements:

Label Encoding: Converts text labels (Luminal A, Luminal B, etc.) to numeric values needed for CNN training
Image Size Analysis: Tracks and reports image dimensions to help you choose an appropriate target size
Preprocessing Function:

Resizes all images to a uniform size (necessary for CNNs)
Normalizes pixel values to [0, 1] range
Optionally applies masks to focus on diseased tissue regions
Returns numpy arrays ready for training


Train/Validation Split: Creates a validation set (20%) while maintaining class distribution for proper model evaluation
Data Shapes: Reports shapes to verify everything is ready for CNN input

# **Normalization**

- Neural networks train better with smaller numbers (typically -1 to +1 or 0 to 1).

- Prevents numerical instability - large values (255) can cause gradient explosions.

- Standard practice - most pre-trained models expect normalized inputs
Faster convergence - optimization algorithms work better.


# Testing

In [5]:
# ------------------ Load test images (NO LABELS) ------------------
print("\nLoading test images...")
dataset_dir = input_dir
test_data_dir  = os.path.join(dataset_dir, "test_data")

# Check test directory - CRITICAL
if not os.path.exists(test_data_dir):
    raise FileNotFoundError(f"CRITICAL ERROR: Test data folder not found at: {test_data_dir}")
print(f"✓ Test folder found: {test_data_dir}")


test_file_list = sorted(os.listdir(test_data_dir))
N = len(test_file_list) // 2  # should be 954


# FAIL-FAST: Check expected file count
if len(test_file_list) != 1908:
    raise ValueError(f"CRITICAL ERROR: Expected {1908} files, found {len(test_file_list)}. Test Dataset is incomplete!")

# First half = images
test_image_files = test_file_list[:N]

# Second half = masks
test_mask_files = test_file_list[N:N*2]

# Initialize lists to store loaded data as numpy arrays
test_images = []      # List of numpy arrays (variable dimensions)
test_masks = []       # List of numpy arrays (variable dimensions)
test_dimensions = []  # List to track (height, width) of each sample

# Track validation issues
failed_samples = []
dimension_mismatches = []


# Iterate through each row in the labels DataFrame
for idx in range(N):
    test_filename_img  = test_image_files[idx]
    test_filename_mask = test_mask_files[idx]

    test_img_path  = os.path.join(test_data_dir, test_filename_img)
    test_mask_path = os.path.join(test_data_dir, test_filename_mask)

    try:
        # Open image and mask
        test_img  = Image.open(test_img_path).convert("RGB")
        test_mask = Image.open(test_mask_path).convert("L")

        # Convert to numpy arrays immediately for memory efficiency
        test_img_array = np.array(test_img)    # Shape: (H, W, 3)
        test_mask_array = np.array(test_mask)  # Shape: (H, W)

        # VALIDATION: Check that image and mask have matching dimensions
        if test_img_array.shape[:2] != test_mask_array.shape:
            dimension_mismatches.append({
                'index': idx,
                'filename_img': test_filename_img,
                'filename_mask': test_filename_mask,
                'img_shape': test_img_array.shape,
                'mask_shape': test_mask_array.shape
            })
            print(f"WARNING: Dimension mismatch at index {idx}: image {test_img_array.shape} vs mask {test_mask_array.shape}")
            continue  # Skip this sample


        # Store numpy arrays instead of PIL images
        test_images.append(test_img_array)
        test_masks.append(test_mask_array)
        test_dimensions.append(test_img_array.shape[:2])


    except Exception as e:
        failed_samples.append({'index': idx, 'error': str(e)})
        print(f"ERROR loading sample index {idx}: {e}")

print(f"\n✓ Successfully loaded {len(test_images)} test samples.")



Loading test images...
✓ Test folder found: /gdrive/My Drive/B University/Artificial Networks/an2dl2526c2/test_data

✓ Successfully loaded 954 test samples.


In [6]:
# ------------------ VALIDATION REPORT ------------------
print("\n" + "="*60)
print("VALIDATION REPORT")
print("="*60)

# Report failed samples
if failed_samples:
    print(f"\n⚠ WARNING: {len(failed_samples)} samples failed to load:")
    for fail in failed_samples[:5]:  # Show first 5
        print(f"  - Index {fail['index']}: {fail['error']}")
    if len(failed_samples) > 5:
        print(f"  ... and {len(failed_samples) - 5} more")

# Report dimension mismatches
if dimension_mismatches:
    print(f"\n⚠ WARNING: {len(dimension_mismatches)} samples had dimension mismatches:")
    for mismatch in dimension_mismatches[:5]:  # Show first 5
        print(f"  - Index {mismatch['index']}: img {mismatch['img_shape']} vs mask {mismatch['mask_shape']}")
    if len(dimension_mismatches) > 5:
        print(f"  ... and {len(dimension_mismatches) - 5} more")

# Check if too many samples failed
total_failed = len(failed_samples) + len(dimension_mismatches)
if total_failed > N * 0.05:  # More than 5% failed
    raise RuntimeError(f"CRITICAL ERROR: Too many samples failed ({total_failed}/{N}). Dataset quality is too low!")


# Display dimension statistics
print(f"\nImage dimension statistics:")
heights = [dim[0] for dim in test_dimensions]
widths = [dim[1] for dim in test_dimensions]
print(f"  Height range: {min(heights)} - {max(heights)} pixels (mean: {np.mean(heights):.1f})")
print(f"  Width range: {min(widths)} - {max(widths)} pixels (mean: {np.mean(widths):.1f})")

# Count unique dimensions
unique_dims = set(test_dimensions)
print(f"  Number of unique dimension combinations: {len(unique_dims)}")
if len(unique_dims) <= 5:
    print(f"  Unique dimensions found:")
    for dim in sorted(unique_dims):
        count = test_dimensions.count(dim)
        print(f"    - {dim}: {count} samples")

print("\n" + "="*60)
print("✓ Training data validation complete!")
print("="*60)


VALIDATION REPORT

Image dimension statistics:
  Height range: 1024 - 4752 pixels (mean: 1323.4)
  Width range: 1024 - 6158 pixels (mean: 1271.3)
  Number of unique dimension combinations: 544

✓ Training data validation complete!
